In [ ]:
!pip install pandas numpy scipy scikit-learn

In [ ]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

#Configurar diretório e carregar dados
!wget http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
!unzip ml-latest-small.zip

dataset_dir = "ml-latest-small"

#Carregar avaliações
ratings = pd.read_csv(f"{dataset_dir}/ratings.csv")  #userId, movieId, rating, timestamp

#Carregar filmes
movies = pd.read_csv(f"{dataset_dir}/movies.csv")    #movieId, title, genres

print("Dados carregados!")
print(f"Total de avaliações: {len(ratings)}")
print(f"Total de filmes: {len(movies)}")

#[v4] NOVO: Criar um mapa de ID -> índice para TODOS os filmes em movies.csv
#Isso é necessário para alinhar a matriz de conteúdo
movie_idx_mapper_full = {mid: i for i, mid in enumerate(movies['movieId'])}

#[v4] NOVO: Vetorizar os Gêneros
#1. Substituir '|' por ' ' para que o TfidfVectorizer trate cada gênero como uma palavra
movies['genres_str'] = movies['genres'].str.replace('|', ' ', regex=False)
#2. Criar a matriz TF-IDF
#min_df=2 ignora gêneros muito raros (ex: "IMAX")
tfidf = TfidfVectorizer(min_df=2)
#genres_matrix_full terá o shape [9742 filmes, N_generos]
genres_matrix_full = tfidf.fit_transform(movies['genres_str'])

--2025-11-04 17:01:52--  http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.96.204
Connecting to files.grouplens.org (files.grouplens.org)|128.101.96.204|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://files.grouplens.org/datasets/movielens/ml-latest-small.zip [following]
--2025-11-04 17:01:52--  https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Connecting to files.grouplens.org (files.grouplens.org)|128.101.96.204|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 978202 (955K) [application/zip]
Saving to: ‘ml-latest-small.zip’

ml-latest-small.zip 100%[===================>] 955.28K  3.30MB/s    in 0.3s    

2025-11-04 17:01:52 (3.30 MB/s) - ‘ml-latest-small.zip’ saved [978202/978202]

Archive:  ml-latest-small.zip
   creating: ml-latest-small/
  inflating: ml-latest-small/links.csv  
  inflating: ml-latest-small/tags.c

In [ ]:
#Mapear IDs para índices e criar matriz esparsa

#O objetivo aqui é transformar os IDs originais (que não são sequenciais)
#em índices contínuos (0, 1, 2, ...) — algo necessário para criar uma matriz.

#Criaremos dicionários de mapeamento de IDs para índices
#Cada linha em ratings representa uma avaliação feita por um usuário a um filme
#Porém os IDs originais em ratings['userId'] e ratings['movieId'] podem não ser sequenciais.
#Para criar matriz users x movies precisamos índices 0..N-1.

#.unique() retorna um array NumPy contendo apenas os valores distintos de uma coluna, na ordem em que aparecem
#enumerate() transforma uma sequência em pares (índice, valor)
'''Se ratings.csv contiver:
   | userId | movieId | rating |
   | ------ | ------- | ------ |
   | 10     | 5       | 3.5    |
   | 42     | 2       | 4.0    |
   | 10     | 7       | 2.0    |
   Então user_mapper = {10: 0, 42: 1}, movie_mapper = {5: 0, 2: 1, 7: 2}
   Usuário 10 vira linha 0, Usuário 42 vira linha 1
   Filme 5 vira coluna 0, Filme 2 vira coluna 1, Filme 7 vira coluna 2
   Matriz esparsa ficará com shape (2, 3) — 2 usuários × 3 filmes.
'''

#Cria mapeamento de cada userId único para um índice sequencial
user_mapper = {uid: i for i, uid in enumerate(ratings['userId'].unique())}
#Cria mapeamento de cada movieId único para um índice sequencial
movie_mapper = {mid: i for i, mid in enumerate(ratings['movieId'].unique())}

#Mapeamento inverso (para retornar IDs originais)
movie_inv_mapper = {i: mid for mid, i in movie_mapper.items()}
user_inv_mapper = {i: uid for uid, i in user_mapper.items()}

#Mapear IDs originais para índices, adiciona colunas com os índices internos
ratings['user_idx'] = ratings['userId'].map(user_mapper)
ratings['movie_idx'] = ratings['movieId'].map(movie_mapper)

#Normalização das notas + ponderação por similaridade
#Usuários têm padrões de avaliação diferentes: uns dão sempre 5 estrelas, outros quase nunca passam de 3.5
#Ao calcular a similaridade, devemos centralizar (subtrair a média de cada usuário)
#Isso remove o “viés do usuário” e aumenta significativamente a qualidade preditiva do modelo
#O modelo se aproxima mais de um Collaborative Filtering baseado em correlação de Pearson

#Calcular média das notas por usuário
user_mean = ratings.groupby('userId')['rating'].mean()
#Subtrair a média de cada usuário
ratings['rating_norm'] = ratings.apply(lambda x: x['rating'] - user_mean.loc[x['userId']], axis=1)

#[v4] NOVO: Alinhar a matriz de gêneros
#N_rated_movies será o N de filmes avaliados (ex: 9724)
N_rated_movies = len(movie_mapper)
#N_features será o N de gêneros únicos
N_features = genres_matrix_full.shape[1]
#Cria uma matriz de zeros com o shape da matriz colaborativa
aligned_genres_matrix = np.zeros((N_rated_movies, N_features))
#Preenche esta matriz...
for mid, idx_rated in movie_mapper.items():
    if mid in movie_idx_mapper_full:
        #Pega o índice do filme na matriz de conteúdo (ex: 0-9741)
        idx_full = movie_idx_mapper_full[mid]
        #Atribui o vetor de gênero (linha) da matriz cheia para a posição correta (idx_rated) na matriz alinhada
        aligned_genres_matrix[idx_rated] = genres_matrix_full[idx_full].toarray()

#Converte para esparsa para o cálculo da similaridade
aligned_genres_matrix_sparse = csr_matrix(aligned_genres_matrix)

#Cria matriz esparsa usuários x filmes
ratings_sparse = csr_matrix(
    (ratings['rating_norm'], (ratings['user_idx'], ratings['movie_idx']))
)

'''Com o exemplo usado antes:
   user_mapper = {10: 0, 42: 1}
   movie_mapper = {5: 0, 2: 1, 7: 2}
   A matriz de usuários × filmes fica:        5     2     7
                                       U10 [ 3.5   0.0   2.0 ]
                                       U42 [ 0.0   4.0   0.0 ]
   Então a representação esparsa (armazenando só valores != 0) fica:
            (0,0)  3.5 --> user 10, movie 5
            (0,2)  2.0 --> user 10, movie 7
            (1,1)  4.0 --> user 42, movie 2
'''

print("Matriz esparsa criada!")
print(f"Shape da matriz usuário-filme: {ratings_sparse.shape}")
print(f"Shape da matriz de gêneros alinhada: {aligned_genres_matrix_sparse.shape}")

Matriz esparsa criada!
Shape da matriz usuário-filme: (610, 9724)
Shape da matriz de gêneros alinhada: (9724, 24)


In [ ]:
#Calcular similaridade entre filmes

#.T = operador de transposição. ratings_sparse tem usuários nas linhas e filmes nas colunas
#Transposição é feita para ter filmes nas linhas (necessário para filme-filme similarity)
#dense_output=False mantém saída esparsa.

#1. Calcular Similaridade Colaborativa (Item-Item CF)
movie_similarity_cf = cosine_similarity(ratings_sparse.T, dense_output=False)

#2. Calcular Similaridade de Conteúdo (Gêneros)
movie_similarity_content = cosine_similarity(aligned_genres_matrix_sparse, dense_output=False)

print("Matrizes de similaridade (Colaborativa e Conteúdo) calculadas!")

Matrizes de similaridade (Colaborativa e Conteúdo) calculadas!


In [ ]:
#NOVA CÉLULA: HIBRIDIZAÇÃO PONDERADA (combinando as matrizes)

#Alpha (α) é o peso
#alpha = 1.0 -> 100% Colaborativo (v3)
#alpha = 0.0 -> 100% Conteúdo

#Um bom ponto de partida é dar mais peso ao colaborativo
alpha = 0.8

#Combinar as matrizes (ambas devem ter o mesmo shape)
hybrid_similarity = (alpha * movie_similarity_cf) + ((1 - alpha) * movie_similarity_content)

print("Matrizes combinadas em uma 'hybrid_similarity'!")

Matrizes combinadas em uma 'hybrid_similarity'!


A operação `hybrid_similarity = (alpha * movie_similarity_cf) + ((1 - alpha) * movie_similarity_content)` é uma soma ponderada (weighted sum). O "pipeline" é uma simples aritmética de matrizes que é possível porque foi feito um trabalho de alinhamento nas células anteriores.
- movie_similarity_cf é uma matriz [9724 filmes x 9724 filmes].
- movie_similarity_content também é uma matriz [9724 filmes x 9724 filmes].

Graças ao `movie_mapper` e ao alinhamento, o filme na linha 50 de ambas as matrizes é o mesmo (ex: "Pulp Fiction"). E o filme na coluna 120 de ambas as matrizes também é o mesmo (ex: "Forrest Gump").

<br> **A lógica de combinação (elemento por elemento):** Vamos imaginar que alpha = 0.8.

Isso significa que estamos dizendo ao modelo: "confie 80% na sabedoria dos usuários (colaborativa) e 20% nas características dos filmes (conteúdo)."

Como exemplo, vamos calcular a nova similaridade híbrida para o nosso par "Pulp Fiction" (linha 50) e "Forrest Gump" (coluna 120):

1. **Pontuação Colaborativa (CF)**
  - O modelo olha `movie_similarity_cf[50, 120]`. O valor é, digamos, 0.3.
  - Significado: usuários que gostaram de Pulp Fiction também gostaram um pouco de Forrest Gump (e vice-versa).
  - Cálculo Ponderado: `0.8 * 0.3 = 0.24`
2. **Pontuação de Conteúdo (Content)**
  - O modelo olha movie_similarity_content[50, 120].
  - "Pulp Fiction" é `Crime|Drama`. "Forrest Gump" é `Comedy|Drama|Romance|War`. Eles compartilham o gênero Drama, mas são bem diferentes no resto.
  - O valor da similaridade de cosseno TF-IDF aqui é, digamos, 0.1. Significado: esses filmes não são muito parecidos em seus gêneros.
3. **Combinação Híbrida**
  - O modelo simplesmente soma os resultados ponderados.
  - `hybrid_similarity[50, 120] = 0.24 + 0.02 = 0.26`

A nova pontuação de similaridade "híbrida" para este par é 0.26.

<br> **Resolvendo o "Cold Start":** agora, imagine um Filme Novo (linha 9000) que acabou de ser adicionado. Ninguém o avaliou ainda.

1. **Pontuação Colaborativa (CF)**
  - `movie_similarity_cf[9000, 50]` (Similaridade entre "Filme Novo" e "Pulp Fiction").
  - Como ninguém avaliou o "Filme Novo", sua linha na `matriz ratings_sparse` é toda de zeros. A similaridade de cosseno com qualquer outro filme será 0.0.
  - Cálculo Ponderado: `0.8 * 0.0 = 0.0`
2. **Pontuação de Conteúdo (Content)**
  - O "Filme Novo" tem os gêneros `Crime|Drama`.
  - O modelo olha `movie_similarity_content[9000, 50]`. Ele compara `Crime|Drama` ("Filme Novo") com `Crime|Drama` ("Pulp Fiction"). A similaridade de cosseno aqui é altíssima, digamos 0.95.
  - Significado: esses filmes são quase idênticos no gênero.
  - Cálculo Ponderado: 0.2 * 0.95 = 0.19
3. **Combinação Híbrida**
  - `hybrid_similarity[9000, 50]` = 0.0 + 0.19 = 0.19

Resultado: mesmo sem nenhuma avaliação, o "Filme Novo" agora tem uma pontuação de similaridade (0.19) com "Pulp Fiction".

<br>**O pipeline:** `alpha` atua como um "botão de volume" para duas fontes de verdade.
- Para filmes com muitos dados, o alpha = 0.8 garante que a opinião dos usuários (CF) domine a recomendação.
- Para filmes novos (sem dados de CF), o componente (1 - alpha) garante que o conteúdo (Content) ainda possa criar similaridades, permitindo que o filme seja recomendado.

In [ ]:
#Seleciona os Top-K vizinhos por item, ignorando similaridades negativas

K = 30  #pode ser alterado livremente

#Converte a matriz de similaridade para formato CSR para operações rápidas
#[v4] MODIFICAÇÃO: Ao invés de 'movie_similarity' usamos a 'hybrid_similarity'
movie_similarity_csr = hybrid_similarity.tocsr()

#Inicializa lista para armazenar os índices e valores de similaridade dos Top-K vizinhos
indices_topk = []
values_topk = []

for i in range(movie_similarity_csr.shape[0]):
    #Extrai a linha i (similaridade do filme i com todos os outros)
    row = movie_similarity_csr.getrow(i).toarray().ravel()

    #Zera similaridades negativas para ignorá-las
    row[row < 0] = 0

    #Remove o próprio item (autossimilaridade = 1)
    row[i] = 0

    #Seleciona apenas os índices dos K maiores valores positivos
    topk_idx = np.argsort(row)[-K:][::-1]  #pega os maiores K
    topk_values = row[topk_idx]

    #Remove zeros do resultado (caso o item tenha < K positivos)
    valid_mask = topk_values > 0
    topk_idx = topk_idx[valid_mask]
    topk_values = topk_values[valid_mask]

    indices_topk.append(topk_idx)
    values_topk.append(topk_values)

#Agora, indices_topk[i] contém os índices dos vizinhos positivos mais similares ao item i
#e values_topk[i] contém os respectivos valores de similaridade

**K=10** --> "Para recomendar um filme, olhe apenas para os 10 filmes mais parecidos que o usuário já avaliou."
   <br>Recomendações mais "de nicho" e específicas.
   <br>Se o usuário gostou de um filme B muito particular, e o filme A é um dos K vizinhos mais próximos de B, ele será fortemente recomendado.

**K=20** --> "Olhe para os 20 filmes mais parecidos."
   <br>Um bom equilíbrio. Ainda obtém-se recomendações específicas, mas suaviza um pouco o ruído (evita que uma única avaliação estranha domine a recomendação).

**K=40 ou K=50** --> "Olhe para um grupo maior de vizinhos."
   <br>As recomendações tendem a ser mais "seguras" e talvez mais "populares" ou genéricas.
   <br>O modelo considera mais evidências, o que dilui o impacto de vizinhos muito específicos.

In [ ]:
#Função para recomendar filmes similares
#Retorna os top_n filmes mais similares a um movie_id dado
#Usa apenas os vizinhos positivos mais similares do filme como base de recomendação -- ou seja, substitui a parte onde pega todos os scores pela lista indices_topk[movie_idx].
def recommend_movies(movie_id, top_n):
    #Verifica se o filme existe no mapeamento
    if movie_id not in movie_mapper:
        print(f"Filme {movie_id} não encontrado.")
        return []

    #Obtém o índice interno do filme
    movie_idx = movie_mapper[movie_id]

    #Obtém vizinhos mais similares pré-calculados (Top-K)
    topk_indices = indices_topk[movie_idx]
    topk_values = values_topk[movie_idx]

    #Cria uma série (índice = índice do filme, valor = similaridade)
    sim_series = pd.Series(topk_values, index=topk_indices)

    #Seleciona os Top N (ou menos, se houver menos que N vizinhos)
    top_indices = sim_series.sort_values(ascending=False).iloc[:top_n].index

    #Monta lista de recomendações com títulos e similaridades
    recommendations = []
    for idx in top_indices:
        mid = movie_inv_mapper[idx]                                   #Converte índice interno de volta para movieId original
        title = movies.loc[movies.movieId == mid, 'title'].values[0]  #Busca título do filme
        score = sim_series[idx]                                       #Similaridade calculada
        recommendations.append((title, score))

    return recommendations

In [ ]:
#Testar recomendações para um filme

#Exemplo: movieId = 1 (Toy Story 1995)
movie_id = 1
top_n=15
top_movies = recommend_movies(movie_id, top_n)

print(f"Top {top_n} filmes similares a '{movies.loc[movies.movieId==movie_id,'title'].values[0]}':")
for title, score in top_movies:
    print(f"{title} — similaridade {score:.3f}")

Top 15 filmes similares a 'Toy Story (1995)':
Toy Story 2 (1999) — similaridade 0.523
Toy Story 3 (2010) — similaridade 0.426
Aladdin (1992) — similaridade 0.406
Finding Nemo (2003) — similaridade 0.386
Incredibles, The (2004) — similaridade 0.382
Wallace & Gromit: The Wrong Trousers (1993) — similaridade 0.379
Monsters, Inc. (2001) — similaridade 0.378
Wallace & Gromit: A Close Shave (1995) — similaridade 0.330
Shrek (2001) — similaridade 0.321
For the Birds (2000) — similaridade 0.308
Up (2009) — similaridade 0.307
Pete's Dragon (2016) — similaridade 0.302
Batman: Mask of the Phantasm (1993) — similaridade 0.302
Charlie Brown Christmas, A (1965) — similaridade 0.298
Lion King, The (1994) — similaridade 0.298


In [ ]:
#Função para recomendar para um usuário
#Retorna recomendações para um usuário baseado em ratings passados e similaridade filme-filme
def recommend_for_user(user_id, top_n=5):
    #Obter índice do usuário
    if user_id not in user_mapper:
        print(f"Usuário {user_id} não encontrado.")
        return []

    #Converte user_id original para o índice interno da matriz
    user_idx = user_mapper[user_id]

    #Filmes avaliados pelo usuário (linha do usuário na matriz)
    #Converte a linha esparsa para array e achata para vetor 1D
    user_ratings = ratings_sparse[user_idx, :].toarray().flatten()

    #Pega os índices dos filmes avaliados (notas > 0)
    rated_indices = np.where(user_ratings > 0)[0]

    #Vetores para armazenar soma ponderada dos scores e soma das similaridades
    scores = np.zeros(ratings_sparse.shape[1])
    sim_sums = np.zeros(ratings_sparse.shape[1])

    '''Para cada filme avaliado, acumular score ponderado:
       - pega os vizinhos Top-K desse filme
       - multiplica a similaridade de cada vizinho pela nota do usuário
       - acumula as somas para depois normalizar'''
    for idx in rated_indices:
        sim_indices = indices_topk[idx]   # índices dos vizinhos Top-K
        sim_values = values_topk[idx]     # valores das similaridades Top-K
        rating = user_ratings[idx]        # nota (normalizada) que o usuário deu ao filme

        #Acumula soma ponderada e normalizador
        scores[sim_indices] += sim_values * rating
        sim_sums[sim_indices] += sim_values

    #Evita divisão por zero
    sim_sums[sim_sums == 0] = 1

    #Nota prevista normalizada = soma ponderada / soma das similaridades
    predicted_ratings = scores / sim_sums

    #---------IMPORTANTE: REVERTER NORMALIZAÇÃO PARA ESCALA ORIGINAL---------
    #user_avg = user_mean.loc[user_id]                 #média real do usuário
    #predicted_ratings = predicted_ratings + user_avg  #soma de volta à média

    #Limita à escala válida do MovieLens (0.5 a 5.0)
    #predicted_ratings = np.clip(predicted_ratings, 0.5, 5.0)

    #Remove filmes já avaliados (não queremos recomendá-los novamente)
    predicted_ratings[rated_indices] = 0

    #Seleciona os filmes com maiores notas previstas
    top_indices = np.argsort(predicted_ratings)[::-1][:top_n]

    #Monta a lista final de recomendações (título e nota prevista)
    recommendations = []

    #Obter os filmes que o usuário avaliou (índices) e suas notas normalizadas
    user_ratings_vec = ratings_sparse[user_idx, :].toarray().flatten()
    rated_indices_all = np.where(user_ratings_vec != 0)[0] #Inclui positivos e negativos

    for idx in top_indices:
        mid = movie_inv_mapper[idx]   #converte índice interno para movieId original
        title = movies.loc[movies.movieId == mid, 'title'].values[0]
        pred_score = predicted_ratings[idx]

        evidence = []
        rec_idx = idx #Apenas para clareza (índice do filme recomendado)

        #[v4] NOVO: Encontrar a justificativa/evidência da recomendação
        for rated_idx in rated_indices_all:
            #Pegar a similaridade entre o filme recomendado (rec_idx) e o filme já avaliado (rated_idx)
            sim = movie_similarity_csr[rec_idx, rated_idx]

            #Queremos apenas similaridades positivas
            if sim > 0:
                norm_rating = user_ratings_vec[rated_idx]

                #Focando em "por que você gostou" (nota normalizada positiva)
                if norm_rating > 0:
                    rated_mid = movie_inv_mapper[rated_idx]
                    rated_title = movies.loc[movies.movieId == rated_mid, 'title'].values[0]
                    #Armazenamos o título, a similaridade e a nota (normalizada) dada
                    evidence.append((rated_title, sim, norm_rating))

        #Ordenar as evidências pela maior contribuição (similaridade * nota) ou só similaridade
        evidence.sort(key=lambda x: x[1], reverse=True) #Ordenando por similaridade

        #Adiciona o filme, a nota prevista e as N principais evidências
        recommendations.append((title, pred_score, evidence[:3])) #Pegando as 3 maiores evidências

    return recommendations

In [ ]:
#Testar recomendações para um usuário

#Definir códigos de escape ANSI para facilitar a leitura
BOLD = '\033[1m'
RESET = '\033[0m'

user_id = 1
top_user_movies = recommend_for_user(user_id, top_n=5)

print(f"Top 5 recomendações para o usuário {user_id}:\n")
for title, score, evidence in top_user_movies:
    #Aplicar o negrito apenas ao título
    print(f"{BOLD}{title}{RESET} — score previsto (normalizado): {score:.3f}")

    if evidence:
        print("  ...porque você gostou de:")
        for e_title, e_sim, e_rating in evidence:
            print(f"      - '{e_title}' (Similaridade: {e_sim:.2f})")
    print("\n") #Adiciona um espaço

Top 5 recomendações para o usuário 1:

Railway Children, The (1970) — score previsto (normalizado): 0.634
  ...porque você gostou de:
      - 'Bedknobs and Broomsticks (1971)' (Similaridade: 0.48)
      - 'Live and Let Die (1973)' (Similaridade: 0.47)
      - 'Rescuers, The (1977)' (Similaridade: 0.46)


Pinocchio (2002) — score previsto (normalizado): 0.634
  ...porque você gostou de:
      - 'Goonies, The (1985)' (Similaridade: 0.32)
      - 'Jungle Book, The (1994)' (Similaridade: 0.26)
      - 'Pinocchio (1940)' (Similaridade: 0.19)


Mission, The (1986) — score previsto (normalizado): 0.634
  ...porque você gostou de:
      - 'Henry V (1989)' (Similaridade: 0.44)
      - 'Pink Floyd: The Wall (1982)' (Similaridade: 0.41)
      - 'Mr. Smith Goes to Washington (1939)' (Similaridade: 0.36)


Lost Weekend, The (1945) — score previsto (normalizado): 0.634
  ...porque você gostou de:
      - 'Mr. Smith Goes to Washington (1939)' (Similaridade: 0.47)
      - 'Rocky (1976)' (Similaridade:

# CÉLULAS AUXILIARES

In [ ]:
#CÉLULA DE TESTE PARA INSPECIONAR AVALIAÇÕES DE UM USUÁRIO

def show_user_history(user_id):
    """
    Exibe os filmes avaliados por um usuário e suas respectivas notas.
    """

    #1. Verificar se o usuário existe no dataset original
    if user_id not in user_mapper:
        print(f"Usuário {user_id} não encontrado.")
        return

    #2. Filtrar as avaliações (ratings) para este usuário
    user_ratings = ratings[ratings['userId'] == user_id]

    #3. Juntar (merge) com a tabela de filmes (movies) para obter os títulos
    user_history = user_ratings.merge(movies, on='movieId', how='left')

    #4. Selecionar apenas as colunas de interesse
    user_history = user_history[['title', 'rating', 'rating_norm']]

    #5. Ordenar pela nota (rating) original, da maior para a menor
    user_history = user_history.sort_values(by='rating', ascending=False)

    #6. Exibir os resultados
    print(f"--- Histórico de Avaliações para o Usuário {user_id} ---")
    print(f"(Média de notas deste usuário: {user_mean.loc[user_id]:.2f})\n")

    print(f"{'Título do Filme':<80} | {'Nota Original':<12} | {'Nota Normalizada':<15}")
    print("-" * 120)

    for _, row in user_history.iterrows():
        print(f"{row['title'][:80]:<80} | {row['rating']:<13} | {row['rating_norm']:<15.2f}")

#=======================TESTAR A FUNÇÃO=======================
#Teste com o usuário 442
show_user_history(user_id=442)
print("\n" + "="*120 + "\n")

--- Histórico de Avaliações para o Usuário 442 ---
(Média de notas deste usuário: 1.27)

Título do Filme                                                                  | Nota Original | Nota Normalizada
------------------------------------------------------------------------------------------------------------------------
Jungle Book, The (1994)                                                          | 2.5           | 1.23           
Tootsie (1982)                                                                   | 2.5           | 1.23           
Pretty in Pink (1986)                                                            | 2.0           | 0.73           
Rudy (1993)                                                                      | 2.0           | 0.73           
Boys Don't Cry (1999)                                                            | 2.0           | 0.73           
Me, Myself & Irene (2000)                                                        | 2.0           | 

## O que muda ao usar o Top-K
Na nova versão:
- predicted_ratings = scores / sim_sums;
- scores = soma das similaridades × nota do usuário;
- sim_sums = soma somente dos Top-K vizinhos relevantes;

E como estamos somando K vizinhos com similaridade real (ex: 0.7, 0.6, 0.5), o denominador (sim_sums) tende a ser pequeno (~3-6).

- Isso faz com que o resultado final (predicted_ratings) caia naturalmente para a escala das notas originais (0-5).
- Agora estamos efetivamente prevendo notas normalizadas, e não apenas acumulando força bruta.

Portanto, valores como 1.268 agora significam algo do tipo: “O modelo prevê que o usuário daria nota 1.27 para esse filme.”

**SEM** A NORMALIZAÇÃO DE VOLTA PARA A ESCALA ORIGINAL VÁLIDA DO MOVIELENS, o modelo está prevendo quanto acima ou abaixo da média o usuário tenderia a avaliar, não a nota final ainda.


Tomemos o usuário 18 de exemplo. Top 5 recomendações para o usuário 18 com k=100:
- Alphaville (Alphaville, une étrange aventure de Lemmy Caution) (1965) — score 1.268
- All Dogs Go to Heaven 2 (1996) — score 1.268
- Boiling Point (1993) — score 1.268
- Tender Mercies (1983) — score 1.268
- Stepford Wives, The (1975) — score 1.268

O sistema está prevendo valores em torno de zero:
- 0 → filme médio para o usuário
- +1.0 → filme que ele tenderia a avaliar 1 ponto acima da média
- -1.0 → filme que ele tenderia a avaliar 1 ponto abaixo da média

Ao somar +3.73 (média pessoal dele), obtém-se previsões em torno de 3.73 ± 1, ou seja, na escala original (0.5–5.0).

In [ ]:
#GEPETO: BLOCO GERADO PARA DIAGNÓSTICOS RÁPIDOS

#1) intervalo das notas no dataset original
print("ratings range:", ratings['rating'].min(), ratings['rating'].max())
print("ratings mean overall:", ratings['rating'].mean())

#2) verificar se tem coluna normalizada (rating_norm) e se ratings_sparse usa ela
print("rating_norm present:", 'rating_norm' in ratings.columns)
#Se ratings_sparse foi reconstruída a partir de ratings['rating_norm'], verifique:
example_user = 442
print("média do usuário", example_user, ":", ratings.groupby('userId')['rating'].mean().loc[example_user])
print("user_ratings (sample) for user 18:", ratings_sparse[user_mapper[example_user], :].toarray().flatten()[ :10 ])

#3) estatísticas das similaridades Top-K para um filme exemplo
movie_idx = 2
vals = values_topk[movie_idx] if 'values_topk' in globals() else movie_similarity[movie_idx].toarray().flatten()
print("similarity stats (example movie): min, max, mean, sum:", np.min(vals), np.max(vals), np.mean(vals), np.sum(vals))

#4) rodar recommend_for_user e inspecionar sim_sums / predicted_ratings internamente
user_id = 442
user_idx = user_mapper[user_id]
user_ratings = ratings_sparse[user_idx,:].toarray().flatten()
rated_indices = np.where(user_ratings>0)[0]

#Compute sim_sums & scores as in your Top-K version
scores = np.zeros(ratings_sparse.shape[1])
sim_sums = np.zeros(ratings_sparse.shape[1])
for idx in rated_indices:
    sim_idx = indices_topk[idx]
    sim_val = values_topk[idx]
    scores[sim_idx] += sim_val * user_ratings[idx]
    sim_sums[sim_idx] += sim_val

print("sim_sums stats:", sim_sums.min(), np.percentile(sim_sums,25), np.median(sim_sums), np.mean(sim_sums), sim_sums.max())
pred = scores / np.where(sim_sums==0, 1, sim_sums)
print("predicted ratings stats:", pred.min(), np.percentile(pred,25), np.median(pred), np.mean(pred), pred.max())


ratings range: 0.5 5.0
ratings mean overall: 3.501556983616962
rating_norm present: True
média do usuário 442 : 1.275
user_ratings (sample) for user 18: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
similarity stats (example movie): min, max, mean, sum: 0.31033821444867743 0.4681497005899975 0.3568007229675794 10.704021689027382
sim_sums stats: 0.0 0.0 0.0 0.010872412239452112 0.8970171217860284
predicted ratings stats: 0.0 0.0 0.0 0.017465753692963493 1.225


In [ ]:
#Descobrir o usuário com menor média de avaliações
lowest_user = user_mean.idxmin()       #retorna o userId com menor média
lowest_mean = user_mean.min()          #valor da menor média

print(f"Usuário com menor média: {lowest_user} (média = {lowest_mean:.3f})")

Usuário com menor média: 442 (média = 1.275)


Top 5 recomendações para o usuário 442, com k=100:
- Real Blonde, The (1997) — score 1.225
- Summer Place, A (1959) — score 1.225
- Beyond the Clouds (Al di là delle nuvole) (1996) — score 1.225
- Total Eclipse (1995) — score 1.225
- Final Analysis (1992) — score 1.225

Usuário 442 dá notas extremamente baixas (bem abaixo da média geral ≈ 3.5). Seu vetor de avaliações é muito esparso e de baixo valor, com 9 filmes avaliados.

Durante a predição, como normalizamos as notas (rating_norm = rating - user_mean), esse usuário 442 tem praticamente notas normalizadas próximas de 0.

Isso faz com que o acumulador
`scores[sim_indices] += sim_values * rating`
resulte em valores muito baixos ou quase nulos (porque rating ≈ 0 ou até negativo, e sim_values pequenas).

Na normalização, mesmo após dividir por sim_sums,
- `predicted_ratings = scores / sim_sums`
- `predicted_ratings += user_avg  #soma a média dele (1.275)`

as notas previstas se concentram em torno da média pessoal dele -- ou seja, tudo fica ≈ 1.27.

Mas como aplicamos np.clip(predicted_ratings, 0.5, 5.0) e np.where(...),
ainda podendo haver arredondamentos internos,
o valor final pode ser arredondado ou truncado para algo como 2.5,
especialmente se a matriz de similaridade for muito diluída (top-K pequeno ou falta de vizinhos relevantes).

Além disso, como avaliou poucos filmes (9) com notas baixas (média = 1.275), o modelo tem pouquíssima informação para inferir o gosto dele.

O algoritmo item-based tenta prever algo assim:
> “Se ele gostou de filme A, então talvez também goste de filme B semelhante.”

Mas… se ele não gostou de quase nada, não há ponto de referência positivo.
Logo, o modelo fica meio “sem chão”, e retorna um valor neutro (2.5) porque:
- é o centro da escala (entre 0.5 e 5.0),
- e o cálculo ponderado por similaridade tende a convergir para esse meio termo quando os pesos são fracos ou cancelam entre si.


Sim, escrevi esse textão todo só pra demonstrar como esse comportamento é normal. O modelo está apenas sendo honesto: “Eu não tenho informação suficiente para sugerir algo com confiança, então fico no meio da escala.”

O que observamos aqui é um dos fenômenos mais clássicos da área: o chamado “cold start” (início frio).